In [18]:
%%time
import requests
import re
from lxml import etree
import pymysql

class GovSpider(object):
    def __init__(self):
        self.url = 'http://www.mca.gov.cn/article/sj/xzqh/2019/'
        self.headers = {'User-Agent':'Mozilla/5.0'}
        self.data = []
        self.sheng = []
        self.shi = []
        self.xian = []
        self.db = pymysql.Connect('localhost','root','123456','spider',charset='utf8')
        self.cursor = self.db.cursor()
        
    def get_link(self):
        html = requests.get(url=self.url,headers=self.headers).text
        # xpath 提取最新链接
        xpath_html = etree.HTML(html)
        xpath_bds = '//tr[2]/td[2]/a/@href'
        link_list = xpath_html.xpath(xpath_bds)
        link = link_list[0] if link_list else None
        if link:
            # link为假链接
            two_url = 'http://www.mca.gov.cn' + link
            print(two_url)
            # 通过假链接获取真链接
            two_link = self.get_real_link(two_url)
            self.parse_html(two_link)
        else:
            print('提取失败')
            return 
        
    def get_real_link(self,link):
        html = requests.get(url=link,headers=self.headers).content.decode()
        re_bds = r'window.location.href="(?P<href>.*?)";'
        pattern = re.compile(re_bds,re.S)
        true_url = pattern.findall(html)[0]
        return true_url
    
    # 提取数据
    def parse_html(self,two_url):
        html = requests.get(url=two_url,headers=self.headers).text
        xpath_html = etree.HTML(html)
        # 基准xpath:节点对象列表
        tr_list = xpath_html.xpath('//tr[@height="19"]')
        for tr in tr_list:
            code = tr.xpath('./td[2]/text()')[0].strip()
            city = tr.xpath('./td[3]/text()')[0].strip()
            self.data.append((code,city))
        print('正在保存数据库')
        self.save_mysql()
        
    def save_mysql(self):
        for code,city in self.data:
            code_int = int(code)
            if code_int % 10000 == 0:
                self.sheng.append((code,city))
            elif code_int % 100 == 0:
                self.shi.append((code,city))
            else:
                self.xian.append((code,city))
        self.save_sheng()
        self.save_shi()
        self.save_xian()
        
    def save_sheng(self):
        ins_sheng = 'insert into sheng(sheng_code,sheng_name) values(%s,%s);'
        try:
            self.cursor.executemany(ins_sheng,self.sheng)
            self.db.commit()
            print('省份数据已存入数据库')
        except Exception as e:
            print(e)
            self.db.rollback()
        
        
    def save_shi(self):
        for sheng_code,sheng_name in self.sheng:
#             self.save_shi(code,sheng_name)
            sheng_code_int = int(sheng_code)
            sheng_type = str(sheng_code_int//10000)
            if sheng_name.endswith('市'):
                shi_list = [(code,city,sheng_code) for code,city in self.xian if code.startswith(sheng_type)]
            else:
                shi_list = [(code,city,sheng_code) for code,city in self.shi if code.startswith(sheng_type)]
            shi_insert = 'insert into shi(shi_code,shi_name,s_code) values(%s,%s,%s)'
            try:
                self.cursor.executemany(shi_insert,args=shi_list)
                self.db.commit()
            except Exception as e:
                print(e,'市插入错误')
                self.db.rollback()
        print('城市数据已保存到数据库中')

        
    def save_xian(self):
        for shi_code,shi_name in self.shi:
#             self.save_xian(shi_code,shi_name)
            shi_code_int = int(shi_code) // 100
            shi_type = str(shi_code_int)
            xian_list = [(code,xian,shi_code) for code,xian in self.xian if code.startswith(shi_type)]
            if xian_list:
                xian_insert = 'insert into xian(xian_code,xian_name,s_code) values(%s,%s,%s)'
                try:
                    self.cursor.executemany(xian_insert,xian_list)
                    self.db.commit()
                except Exception as e:
                    print(e,'县插入错误')
                    self.db.rollback()
            else:
                continue 
        print('县数据已保存到数据库中')
    
    def run(self):
        self.get_link()
#         print(self.data)
        
spider = GovSpider()
spider.run()

http://www.mca.gov.cn/article/sj/xzqh/2019/201909/20190900019994.shtml
正在保存数据库
省份数据已存入数据库
城市数据已保存到数据库中
县数据已保存到数据库中
CPU times: user 1.15 s, sys: 155 ms, total: 1.3 s
Wall time: 33.4 s
